# Warping tutorial
## g_filtering_multiples_modes_for_loc

##### May 2020
###### Eva Chamorro - Daniel Zitterbart - Julien Bonnel

## 1. Import packages

In [2]:
import os
import matplotlib
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
%matplotlib widget
from matplotlib import interactive
from matplotlib.path import Path
from scipy import interpolate
from scipy.fftpack import fft, ifft
from scipy.signal import hilbert
from ipywidgets import interact, interact_manual
from warping_functions import *
from time_frequency_analysis_functions import *
from bbox_select import *
from pts_select import *

## 2. Load simulated signal

In [3]:
data = sio.loadmat(os.getcwd()+ '/sig_pek_for_warp.mat')


'''
    s_t: propagated modes in a Pekeris waveguide with parameters
    c1, c2, rho1, rho2: sound speed / density
        D: depth
        r: range
        zs, zr: source/receiver depth
    s_t_dec: same than s_t, except that time origin has been set for warping
    fs: sampling frequency
    
   
     NB: one can run optional_create_simulated_signal.m to generate another
     simulated signal
'''

# Select variables
s_t=data['s_t']
fs=data['fs']
s_t_dec=data['s_t_dec']
r=data['r']
c1=data['c1']
c2=data['c2']
D=data['D']
rho1=data['rho1']
rho2=data['rho2']

## 2. Process signal

In [4]:
# The first sample of s_t_dec corresponds to time r/c1

# Make the signal shorter, no need to keep samples with zeros
N_ok=150
s_ok=s_t_dec[:,0:N_ok] ### this is the impulse response of the waveguide

#Corresponding time and frequency axis
time=np.arange(0,N_ok)/fs

# The warped signal will be s_w
[s_w, fs_w]=warp_temp_exa(s_ok,fs,r,c1)
M=len(s_w)

## 3. Time frequency representations

### 3.1 Original signal

In [5]:
# STFT computation

NFFT=1024
N_window=31 # you need a short window to see the modes
b=np.arange(1,N_ok+1)
b=b[np.newaxis,:]
h=np.hamming(N_window)
h=h[:,np.newaxis]

tfr=tfrstft(s_ok,b,NFFT,h)
spectro=abs(tfr)**2

#Figure
freq=np.arange(0,NFFT)*fs/NFFT

print('This is the spectrogram of the received signal')
print('We will now warp it')

plt.figure(figsize=(15.0,5.0))
plt.subplot(121)
plt.imshow(spectro, extent=[time[0,0], time[0,-1], freq[0,0], freq[0,-1]],aspect='auto', origin='low')
plt.ylim([0, fs/2])
plt.xlim([0,0.5])  ### Adjust this to see better
plt.xlabel('Time (sec)')
plt.ylabel('Frequency (Hz)')
plt.title('Source signal')
plt.show()


print('Continue to warp it')


This is the spectrogram of the received signal
We will now warp it


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Continue to warp it


### 3.2 Warped signal

In [9]:
# STFT computation
N_window_w=301  # You need a long window to see the warped modes
wind=np.hamming(N_window_w)
wind=wind/np.linalg.norm(wind)
wind=wind[:,np.newaxis]
b=np.arange(1,M+1)
b=b[np.newaxis,:]
tfr_w=tfrstft(s_w,b,NFFT,wind)
spectro_w=abs(tfr_w)**2

# Time and frequency axis of the warped signal
time_w=np.arange(0,(M)/fs_w, 1/fs_w)
freq_w=np.arange(0,fs_w-fs_w/NFFT+fs_w/NFFT,fs_w/NFFT)
    
#Figure
plt.figure()
plt.imshow(spectro_w, extent=[time_w[0], time_w[-1], freq_w[0], freq_w[-1]] ,aspect='auto', origin='low')
plt.ylim([0,40]) ### Adjust this to see better
plt.xlabel('Warped time (sec)')
plt.ylabel('Corresponding warped frequency (Hz)')
plt.title('Warped signal')
plt.show()
 
print('Continue ')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Continue 


## 4. Filtering

In [10]:
# To make it easier, filtering will be done by hand using bbox_select.
# See bbox_selec.py for more information

spectro_w_1=spectro_w[0:200,:]
Nmode=4
modes=np.zeros((N_ok,Nmode))
tm=np.zeros((NFFT,Nmode))

print('Now try to filter the 4 modes')
print('Create the four masks sequentially, starting with mode 1, then 2, etc.')
print('To do so, click twice n the spectrogram to create a line and continue to define the region you want to filter ')
print('(if needed, go back to c_warping_and_filtering.m for mask creation)')
print('Contine to filter mode 1')

Now try to filter the 4 modes
Create the four masks sequentially, starting with mode 1, then 2, etc.
To do so, click twice n the spectrogram to create a line and continue to define the region you want to filter 
(if needed, go back to c_warping_and_filtering.m for mask creation)
Contine to filter mode 1


#### Filter mode 1

In [11]:
# Let filter a mode 1
print('Filter mode 1')
interactive(True)
section=bbox_select(spectro_w_1)
print('When you have finish,click "disconnect mpl" and continue to filter mode 2') 

Filter mode 1


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Button(description='Disconnect mpl', style=ButtonStyle())

When you have finish,click "disconnect mpl" and continue to filter mode 2


#### Filter mode 2

In [12]:
# create the mask of the section
pts=section.selected_points
nx, ny = np.shape(spectro_w_1)
x, y = np.meshgrid(np.arange(ny), np.arange(nx))
x, y = x.flatten(), y.flatten()
points = np.vstack((x,y)).T
path = Path(pts)
grid = path.contains_points(points)
mask = grid.reshape((nx,ny))

masque_1=np.double(mask)

# add the part masked to the total sprectogram array
masque_0=np.zeros_like(spectro_w[200:,:])
masque=np.concatenate((masque_1,masque_0),axis=0)


mode_rtf_warp=masque*tfr_w
norm=1/NFFT/np.max(wind)
mode_temp_warp=np.real(np.sum(mode_rtf_warp,axis=0))*norm*2
mode=iwarp_temp_exa(mode_temp_warp,fs_w,r,c1,fs,N_ok)
modes[:,0]=mode[:,0]

## Verification

a=hilbert(mode)
b=np.arange(1,N_ok+1)
b=b[np.newaxis,:]
h=np.hamming(N_window)
h=h[:,np.newaxis]
mode_stft=tfrstft(a,b,NFFT,h)
mode_spectro=abs(mode_stft)**2
tm_1,D2=momftfr(mode_spectro,0,N_ok,time)
tm[:,0]=tm_1[:,0]

## Mode 2
print('Filter mode 2')
interactive(True)
section_2=bbox_select(spectro_w_1)
print('When you have finish,click "disconnect mpl" and continue to filter mode 3') 

Filter mode 2


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Button(description='Disconnect mpl', style=ButtonStyle())

When you have finish,click "disconnect mpl" and continue to filter mode 3


#### Filter mode 3

In [13]:
# create the mask of the section
pts=section_2.selected_points
nx, ny = np.shape(spectro_w_1)
x, y = np.meshgrid(np.arange(ny), np.arange(nx))
x, y = x.flatten(), y.flatten()
points = np.vstack((x,y)).T
path = Path(pts)
grid = path.contains_points(points)
mask = grid.reshape((nx,ny))

masque_2=np.double(mask)

# add the part masked to the total sprectogram array
masque_2=np.concatenate((masque_2,masque_0),axis=0)


mode_rtf_warp_2=masque_2*tfr_w
mode_temp_warp_2=np.real(np.sum(mode_rtf_warp_2,axis=0))*norm*2
mode_2=iwarp_temp_exa(mode_temp_warp_2,fs_w,r,c1,fs,N_ok)
modes[:,1]=mode_2[:,0]

## Verification
a_2=hilbert(mode_2)
mode_stft_2=tfrstft(a_2,b,NFFT,h)
mode_spectro_2=abs(mode_stft_2)**2
tm_2,D2_2=momftfr(mode_spectro_2,0,N_ok,time)
tm[:,1]=tm_2[:,0]

## Mode 3
print('Filter mode 3')
interactive(True)
section_3=bbox_select(spectro_w_1)
print('When you have finish,click "disconnect mpl" and continue to filter mode 4') 

Filter mode 3


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Button(description='Disconnect mpl', style=ButtonStyle())

When you have finish,click "disconnect mpl" and continue to filter mode 4


#### Filter mode 4

In [14]:
# create the mask of the section
pts=section_3.selected_points
nx, ny = np.shape(spectro_w_1)
x, y = np.meshgrid(np.arange(ny), np.arange(nx))
x, y = x.flatten(), y.flatten()
points = np.vstack((x,y)).T
path = Path(pts)
grid = path.contains_points(points)
mask = grid.reshape((nx,ny))

masque_3=np.double(mask)

# add the part masked to the total sprectogram array
masque_3=np.concatenate((masque_3,masque_0),axis=0)


mode_rtf_warp_3=masque_3*tfr_w
mode_temp_warp_3=np.real(np.sum(mode_rtf_warp_3,axis=0))*norm*2
mode_3=iwarp_temp_exa(mode_temp_warp_3,fs_w,r,c1,fs,N_ok)
modes[:,2]=mode_3[:,0]

## Verification

a_3=hilbert(mode_3)
mode_stft_3=tfrstft(a_3,b,NFFT,h)
mode_spectro_3=abs(mode_stft_3)**2
tm_3,D2_3=momftfr(mode_spectro_3,0,N_ok,time)
tm[:,2]=tm_3[:,0]

## Mode 4
print('Filter mode 4')
interactive(True)
section_4=bbox_select(spectro_w_1)
print('When you have finish,click "disconnect mpl" and continue') 

Filter mode 4


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Button(description='Disconnect mpl', style=ButtonStyle())

When you have finish,click "disconnect mpl" and continue


In [16]:
# create the mask of the section
pts=section_4.selected_points
nx, ny = np.shape(spectro_w_1)
x, y = np.meshgrid(np.arange(ny), np.arange(nx))
x, y = x.flatten(), y.flatten()
points = np.vstack((x,y)).T
path = Path(pts)
grid = path.contains_points(points)
mask = grid.reshape((nx,ny))
masque_4=np.double(mask)

# add the part masked to the total sprectogram array
masque_4=np.concatenate((masque_4,masque_0),axis=0)


mode_rtf_warp_4=masque_4*tfr_w
mode_temp_warp_4=np.real(np.sum(mode_rtf_warp_4,axis=0))*norm*2
mode_4=iwarp_temp_exa(mode_temp_warp_4,fs_w,r,c1,fs,N_ok)
modes[:,3]=mode_4[:,0]

## Verification

a_4=hilbert(mode_4)
mode_stft_4=tfrstft(a_4,b,NFFT,h)
mode_spectro_4=abs(mode_stft_4)**2
tm_4,D2=momftfr(mode_spectro_4,0,N_ok,time)
tm[:,3]=tm_4[:,0]

print('End of filtering')
print('Continue')

End of filtering
Continue


## 5. Verification

In [17]:
print('The red lines are the estimated dispersion curves.')
print('Now let us restrict them to a frequency band where they are ok')
print('You will have to enter the min/max frequency for each dispersion curves')
print('(for every mode, choose the widest frequency band over which the dispersion curve estimation looks correct)')


plt.figure()
plt.imshow(spectro, extent=[time[0,0], time[0,-1], freq[0,0], freq[0,-1]],aspect='auto',origin='low')
plt.ylim([0,fs/2])
plt.xlabel('Time (sec)')
plt.ylabel('Frequency (Hz)')
plt.title('Spectrogram and estimated dispersion curves')
plt.plot(tm[:,0],freq[0, :],'r')
plt.plot(tm[:,1],freq[0, :],'r')
plt.plot(tm[:,2],freq[0, :],'r')
plt.plot(tm[:,3],freq[0, :],'r')

print('Continue to select the frequency band')



The red lines are the estimated dispersion curves.
Now let us restrict them to a frequency band where they are ok
You will have to enter the min/max frequency for each dispersion curves
(for every mode, choose the widest frequency band over which the dispersion curve estimation looks correct)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Continue to select the frequency band


## 6. Restrict to frequency band of interest

In [18]:
fmin=np.zeros(Nmode)
fmax=np.zeros(Nmode)
for i in range (Nmode):
    fmin[i]= int(input('Enter min freq for mode '+str(i+1)+':  '))
    fmax[i]= int(input('Enter max freq for mode '+str(i+1)+':  '))


tm_ok=tm
freq_sel=freq[0,:]

for i in range(Nmode):
    pos=np.where((freq_sel > fmin[i]) & (freq_sel < fmax[i]))
    poss=np.array(pos,dtype='int')
    tm_ok[0:poss[0,0],i]=np.nan
    tm_ok[poss[0,-1]:,i]=np.nan

    
print('This is the spectrogram with your best guess of the dispersion curves.')
plt.figure()
plt.imshow(spectro, extent=[time[0,0], time[0,-1], freq[0,0], freq[0,-1]],aspect='auto',origin='low')
plt.ylim([0, fs/2])
plt.xlabel('Time (sec)')
plt.ylabel('Frequency (Hz)')
plt.title('Spectrogram and estimated dispersion curves')
plt.plot(tm_ok[:,0],freq[0, :],'r')
plt.plot(tm_ok[:,1],freq[0, :],'r')
plt.plot(tm_ok[:,2],freq[0, :],'r')
plt.plot(tm_ok[:,3],freq[0, :],'r')
plt.show()


r_true=r
c1_true=c1
c2_true=c2
rho1_true=rho1
rho2_true=rho2
D_true=D

### let's introduce a small random time shift, as if source and receiver
### were not synchronized

dt_true=np.random.rand(1)*0.06-0.03
tm_for_inv=tm_ok+dt_true

### let's save the tm_for_inv on a relevant frequency axis

fmin_data=min(fmin)
fmax_data=max(fmax)
df_data=2
freq_data=np.arange(fmin_data,fmax_data+df_data,df_data)
Nf_data=len(freq_data)

data=np.zeros((Nf_data, Nmode))

for i in range (Nmode):
    f = interpolate.interp1d(freq[0,:], tm_for_inv[:,i], bounds_error=False, fill_value=np.nan )
    data[:,i] = f(freq_data)
    
sio.savemat('save.mat',{'r_true': r_true, 'c1_true': c1_true,'c2_true':c2_true,'rho1_true':rho1_true,
                        'rho2_true':rho2_true,'D_true':D_true,'freq_data':freq_data,'data':data,'dt_true':dt_true })



print('Your result is saved. If you are happy with it, ')
print('Proceed to the next code for source localization.')
print(' ')
print('END')


Enter min freq for mode 1:   10
Enter max freq for mode 1:   90
Enter min freq for mode 2:   20
Enter max freq for mode 2:   90
Enter min freq for mode 3:   10
Enter max freq for mode 3:   90
Enter min freq for mode 4:   40
Enter max freq for mode 4:   90


This is the spectrogram with your best guess of the dispersion curves.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Your result is saved. If you are happy with it, 
Proceed to the next code for source localization.
 
END
